In [36]:

from DSSATTools import (
    crop, # module that contains the classes for the different crops.
    WeatherStation, # class needed to define the Weather. 
    SoilProfile, SoilLayer, # classes that define the entire soil profile, and the soil layers within the soil profile
    filex, # module that contains the clases for the FileX sections.
    DSSAT # the simulation environment class.
)
from DSSATTools.filex import IrrigationEvent, FertilizerEvent, MowEvent # Individual events
import pandas as pd
from datetime import datetime
import tempfile
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import os
import shutil
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import Optional, Dict
import logging
import sys
import time
from typing import List

In [ ]:

# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"
OUTPUT_FILENAME = "MY_GENERATED_SOIL22.SOL"
START_DATE = "2014-01-01"
END_DATE = "2014-12-31"
PLANTING_DATE = datetime(2014, 6, 2)
API_KEY = "eddbc7c0c9e63e225934add809624c6e" # This not orginal key
SOIL_ID = f"SG{str(int(abs(LATITUDE*100))):0>4}{str(int(abs(LONGITUDE*100))):0>4}"
WEATHER_FILE = './weather/CHEN1401.WTH'
SOIL_FILE = './MY_GENERATED_SOIL22.SOL'
#---------------------------------

In [38]:
# creating a Weather file  from a realtime source

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Fetches O3 data from OpenWeatherMap API."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("Warning: OpenWeatherMap API key is missing.")
        return pd.DataFrame()
    
    print("Fetching OpenWeatherMap O3 data...")
    try:
        start_ts = int(pd.to_datetime(start_str).timestamp())
        end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()
        data = response.json().get('list', [])

        if not data:
            print("...O3 data fetched, but no records returned for the period.")
            return pd.DataFrame()

        daily_data = defaultdict(list)
        for entry in data:
            local_date = datetime.fromtimestamp(entry['dt']).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
        df = pd.DataFrame(records)
        df['date'] = pd.to_datetime(df['date'])
        print("...O3 data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching O3 data: {e}. Proceeding without it.")
        return pd.DataFrame()

def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    """Fetches NASA POWER weather data."""
    print("Fetching NASA POWER data...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        df = df.replace([-999, -99], np.nan)
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
            "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
            "RH2M": "rhum", "EVPTRNS": "evap"
        })
        print("...NASA data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"FATAL: Error fetching NASA POWER data: {e}")
        return pd.DataFrame()

def create_weather_file(
    api_key: str,
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    site_name: str = "SITE",
    insi: str = "XXXX",
    elev: Optional[float] = None
) -> Optional[pd.DataFrame]:
    """
    Fetches weather data and writes a standard DSSAT-compatible .WTH file.
    """
    weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
    if weather_df.empty:
        print("Could not fetch base weather data from NASA POWER. Aborting.")
        return None

    o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
    df = weather_df
    if not o3_df.empty:
        df = pd.merge(df, o3_df, on='date', how='left')

    if 'ozon7' in df.columns:
        df['ozon7'] = df['ozon7'].fillna(50.0)
    else:
        df['ozon7'] = 50.0
    
    df['dco2'] = 421.0

    df["wind"] = df["wind"] * 86.4
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
    # --- CHANGE #1: Revert to the standard YRDOY (YYYYJJJ) format ---
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    df_out = df.fillna(-99)
    
    # --- CHANGE #2: Update the list of columns to use YRDOY ---
    output_cols = ["YRDOY","srad","tmax","tmin","rain","rhum","wind","dewp","par","vapr","dco2","ozon7","evap"]
    
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    
    year_2_digit = pd.to_datetime(start_date).strftime('%y')
    wth_file = os.path.join(output_dir, f"{insi}{year_2_digit}01.WTH")
    WEATHER_FILE = wth_file
    print(f"\nWriting weather data to: {wth_file}")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},NASA\n\n")
        f.write("@ INSI      LAT     LONG    ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else -99:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
        f.write("@DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        
        # --- CHANGE #3: Write the date as a 7-digit integer ---
        for _, row in df_out[output_cols].iterrows():
            f.write(f"{row.YRDOY:>5s}"  # ensure "24001", not 24001
                    f"{row.srad:6.1f}{row.tmax:6.1f}{row.tmin:6.1f}{row.rain:6.1f}"
                    f"{row.rhum:6.1f}{row.wind:6.1f}{row.dewp:6.1f}{row.par:6.1f}"
                    f"{row.vapr:6.3f}{row.dco2:6.1f}{row.ozon7:6.0f}{row.evap:6.1f}\n")
    
    print("✅ File created successfully.")
    return df_out

if __name__ == "__main__":
    final_data = create_weather_file(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        site_name="CHENNAI",
        insi="CHEN",
        elev=6.0
    )

Fetching NASA POWER data...
...NASA data fetched successfully.
Fetching OpenWeatherMap O3 data...
...O3 data fetched, but no records returned for the period.

Writing weather data to: weather\CHEN1401.WTH
✅ File created successfully.


In [39]:
# Creating Soil profile from realtime 

# Import the necessary classes AND the helper function from DSSATTools
from DSSATTools import SoilProfile, SoilLayer
from DSSATTools.soil import estimate_from_texture

# --- 2. HELPER FUNCTIONS ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_and_estimate_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data and uses the DSSATTools `estimate_from_texture` 
    function to calculate hydraulic properties for each layer.
    """
    logging.info("Processing raw data and estimating hydraulic properties with DSSATTools...")
    
    dssat_df = pd.DataFrame(index=raw_df.index)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    
    for index, row in raw_df.iterrows():
        slcl = row['clay'] / 10
        slsi = row['silt'] / 10
        sbdm = row['bdod'] / 100
        
        dssat_df.loc[index, 'SLCL'] = slcl
        dssat_df.loc[index, 'SLSI'] = slsi
        dssat_df.loc[index, 'SBDM'] = sbdm

        if pd.notna(slcl) and pd.notna(slsi) and pd.notna(sbdm):
            estimated_props = estimate_from_texture(slcl=slcl, slsi=slsi, sbdm=sbdm)
            dssat_df.loc[index, 'SLLL'] = estimated_props['slll']
            dssat_df.loc[index, 'SDUL'] = estimated_props['sdul']
            dssat_df.loc[index, 'SSAT'] = estimated_props['ssat']
            dssat_df.loc[index, 'SSKS'] = estimated_props['ssks']
        else:
            logging.warning(f"Missing texture/bulk density for layer {index}. Cannot estimate properties.")

    return dssat_df

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file_with_dssattools(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
    """
    Builds a SoilProfile object from the DataFrame and writes it to a file
    using the DSSATTools library classes.
    """
    logging.info(f"Building SoilProfile object with DSSATTools...")
    
    soil_layers = []
    for _, row in df.iterrows():
        kwargs = {k.lower(): v for k, v in row.to_dict().items()}
        kwargs = {k: v for k, v in kwargs.items() if pd.notna(v)}
        soil_layers.append(SoilLayer(**kwargs))

    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100))):0>4}{str(int(abs(lon*100))):0>4}"
    
    # --- START OF SLDR FIX ---
    # 1. Create a lookup table for SLDR based on soil texture class.
    # These values are based on typical ranges found in the SOIL.SOL reference file.
    SLDR_LOOKUP = {
        'S': 0.75, 'LS': 0.70, 'SL': 0.65,  # Sandy soils (high drainage)
        'L': 0.60, 'SIL': 0.55, 'SI': 0.55,   # Loam soils (medium drainage)
        'SCL': 0.50, 'CL': 0.50, 'SICL': 0.45, # Clay Loam soils
        'SC': 0.40, 'SIC': 0.35, 'C': 0.30,   # Clay soils (low drainage)
    }

    # 2. Look up the sldr_value, with a safe default of 0.5
    sldr_value = SLDR_LOOKUP.get(texture, 0.5)
    logging.info(f"Determined soil texture as '{texture}', setting SLDR to {sldr_value}")
    # --- END OF SLDR FIX ---
    
    soil_profile = SoilProfile(
        name=soil_id,
        soil_series_name='SoilGrids Auto-Generated Profile',
        soil_clasification=texture,
        site='SoilGrids',
        country=country,
        lat=lat,
        long=lon,
        salb=0.13, slu1=6.0, slro=75.0, 
        sldr=sldr_value, # <-- 3. Use the dynamically determined sldr_value instead of -99
        slnf=1.0, slpf=1.0,
        table=soil_layers
    )
    
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    with open(filename, 'w') as f:
        f.write(soil_profile._write_sol())

# --- 3. MAIN EXECUTION LOGIC ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    try:
        raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
        
        final_dssat_profile = process_and_estimate_dssat_format(raw_soil_data)

        print("\n--- Final Processed DSSAT Data ---")
        print(final_dssat_profile.round(3).to_string())

        write_sol_file_with_dssattools(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
        print(f"\n✅ Successfully created soil file using DSSATTools: {OUTPUT_FILENAME}")

        print(f"\nVerifying '{OUTPUT_FILENAME}'...")
        soil_id_to_check = SOIL_ID
        verified_soil = SoilProfile.from_file(soil_id_to_check, OUTPUT_FILENAME)
        print("✅ Verification successful! The file can be read by DSSATTools.")

    except Exception as e:
        logging.error(f"Script failed: {e}")


2025-10-13 01:26:01,567 - INFO - Fetching SoilGrids data for lat=12.3811, lon=78.9366
2025-10-13 01:26:01,567 - INFO - --> Fetching property: sand
2025-10-13 01:26:04,252 - INFO - --> Fetching property: silt
2025-10-13 01:26:06,813 - INFO - --> Fetching property: clay
2025-10-13 01:26:09,577 - INFO - --> Fetching property: bdod
2025-10-13 01:26:12,344 - INFO - --> Fetching property: cfvo
2025-10-13 01:26:15,417 - INFO - --> Fetching property: wv0033
2025-10-13 01:26:18,502 - INFO - --> Fetching property: wv1500
2025-10-13 01:26:21,354 - INFO - --> Fetching property: soc
2025-10-13 01:26:24,325 - INFO - --> Fetching property: phh2o
2025-10-13 01:26:27,295 - INFO - --> Fetching property: cec
2025-10-13 01:26:29,157 - WARNING - Rate limit hit for 'cec'. Retrying in 5s...
2025-10-13 01:26:37,021 - INFO - --> Fetching property: nitrogen
2025-10-13 01:26:40,095 - INFO - Processing raw data and estimating hydraulic properties with DSSATTools...

--- Final Processed DSSAT Data ---
           S

In [40]:
weather_station =  WeatherStation.from_files([WEATHER_FILE])

soil_profile = SoilProfile.from_file(SOIL_ID, SOIL_FILE)

cultivar = crop.Sorghum('IB0026')

field = filex.Field(id_field='REALTIME', wsta=weather_station, id_soil=soil_profile)

planting = filex.Planting(pdate=PLANTING_DATE, ppop=18, ppoe=18, plrs=45, pldp=5)

soil_depths = [int(layer['slb']) for layer in soil_profile.table]

# MODIFIED: Reduce initial soil water and use realistic irrigation schedule
initial_values_table = [(depth, 0.20, 1.5, 1.5) for depth in soil_depths]  # Reduced from 0.35 to 0.20
initial_conditions = filex.InitialConditions(
    pcr='SG', icdat=PLANTING_DATE, icres=1300, icren=0.5,
    table=pd.DataFrame(initial_values_table, columns=['icbl', 'sh2o', 'snh4', 'sno3'])
)

simulation_controls = filex.SimulationControls(
    general=filex.SCGeneral(sdate=PLANTING_DATE),
    options=filex.SCOptions(water='Y', nitro='N', symbi='N'),
    methods=filex.SCMethods(infil='S'),
    management=filex.SCManagement(irrig='R', ferti='N', resid='N', harvs='M'),
    outputs=filex.SCOutputs(grout='Y', waout='Y', niout='Y',ovvew='Y')
)

IR001	On Reported Dates	Applies a specific amount of water (IRAMT) on the exact dates you list in the file.

In [41]:
# irrigation_events = [
#     filex.IrrigationEvent(idate=PLANTING_DATE + timedelta(days=day), 
#                          irop='IR001', irval=15) 
#     for day in range(10, 90, 10)  # Every 10 days, 15mm each
# ]
irrigation_events = [filex.IrrigationEvent(idate=PLANTING_DATE , irop="IR001", irval=10)]

In [42]:

# --- 2. RUN SIMULATION ---
irrigation = filex.Irrigation(
    efir=1, idep=30, iame='IR001',
    table=irrigation_events)
dssat_output = None
try:
    workspace = os.path.join("dummy", "dssat_realistic_run")
    os.makedirs(workspace, exist_ok=True)
    
    shutil.copy(WEATHER_FILE, workspace)
    shutil.copy(SOIL_FILE, workspace)
    
    dssat = DSSAT(workspace)
    print(f"🚀 Running simulation in: {workspace}")
    
    results = dssat.run_treatment(
        field=field, cultivar=cultivar, planting=planting,
        initial_conditions=initial_conditions, irrigation=irrigation,
        simulation_controls=simulation_controls
    )
    print("✅ Simulation completed successfully.")
    dssat_output = dssat.output_tables

except Exception as e:
    print(f"❌ An error occurred during the DSSAT run: {e}")

dummy\dssat_realistic_run created.
🚀 Running simulation in: dummy\dssat_realistic_run

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  59  97  2473   593   212    10   221     0     0   -99      0  187
✅ Simulation completed successfully.


In [43]:
# create daily Precitpation because in output we can only see the cumulative 
dssat_output['SoilWat']['DPREC'] = dssat_output['SoilWat']['PREC'].diff().fillna(
    dssat_output['SoilWat']['PREC'].iloc[0]
)

In [44]:

dssat_output['SoilWat']['Rainfall7DaySum'] = dssat_output['SoilWat']['DPREC'].rolling(window=7, min_periods=1).sum()
dssat_output['SoilWat']['RainTodayFlag'] = dssat_output['SoilWat']['DPREC'].apply(lambda x: 1 if x > 0 else 0)
# Initialize list to store days since last rain
days_since_last_rain_list = []

counter = None
for rain_flag in dssat_output['SoilWat']['RainTodayFlag']:
    if rain_flag == 1:
        counter = 0
    else:
        counter = counter + 1 if counter is not None else None
    days_since_last_rain_list.append(counter)

# Assign list to new column
dssat_output['SoilWat']['DaysSinceLastRain'] = days_since_last_rain_list

# Drop helper column
dssat_output['SoilWat'].drop(columns='RainTodayFlag', inplace=True)

In [45]:
current_day = 2

current_weather =  dssat_output['Weather'][dssat_output['Weather']['DAS'] == current_day]
current_plantgro =  dssat_output['PlantGro'][dssat_output['PlantGro']['DAS'] == current_day]
current_soilorg =  dssat_output['SoilOrg'][dssat_output['SoilOrg']['DAS'] == current_day]
current_soilwat =  dssat_output['SoilWat'][dssat_output['SoilWat']['DAS'] == current_day]
sw_cols = [col for col in dssat_output['SoilWat'].columns if re.match(r'^SW\d+D?$', col)]

def get_sw_by_depth(l,h):
    results = []
    for a,b in zip(sw_cols, soil_depths):
        if b > l and b <= h:
            results.append(current_soilwat[a])
    return (sum(results)/len(results)).iloc[0]


In [46]:
# Suppose idate is DateType
idate = irrigation_events[-1].get('idate')

# Convert to datetime
if isinstance(idate, pd._libs.tslibs.timestamps.Timestamp):  # Pandas Timestamp
    idate_dt = idate.to_pydatetime()
else:  # DSSAT DateType or similar
    idate_dt = datetime(idate.year, idate.month, idate.day)

# Now the operation works
days_since_last_irrigation = ((PLANTING_DATE + timedelta(days=current_day)) - idate_dt).days

In [47]:
print(
f"current_weather : {current_weather.columns}",
f"current_soilwat : {current_soilwat.columns}",
f"current_plantgro : {current_plantgro.columns}",
f"current_soilorg : {current_soilorg.columns}",
sep='\n\n')

current_weather : Index(['@YEAR', 'DOY', 'DAS', 'PRED', 'DAYLD', 'TWLD', 'SRAD', 'PARD', 'CLDD',
       'TMXD', 'TMND', 'TAVD', 'TDYD', 'TDWD', 'TGAD', 'TGRD', 'WDSD', 'CO2D',
       'VPDF', 'VPD', 'OZON7', 'WDATE'],
      dtype='object')

current_soilwat : Index(['@YEAR', 'DOY', 'DAS', 'SWTD', 'SWXD', 'ROFC', 'DRNC', 'PREC', 'IR#C',
       'IRRC', 'DTWT', 'MWTD', 'TDFD', 'TDFC', 'ROFD', 'SW1D', 'SW2D', 'SW3D',
       'SW4D', 'SW5D', 'SW6D', 'SW7D', 'SW8D', 'SW9D', 'SW10', 'DPREC',
       'Rainfall7DaySum', 'DaysSinceLastRain'],
      dtype='object')

current_plantgro : Index(['@YEAR', 'DOY', 'DAS', 'DAP', 'L#SD', 'GSTD', 'LAID', 'LWAD', 'SWAD',
       'GWAD', 'RWAD', 'VWAD', 'CWAD', 'G#AD', 'GWGD', 'HIAD', 'PWAD', 'P#AD',
       'WSPD', 'WSGD', 'NSTD', 'EWSD', 'PST1A', 'PST2A', 'KSTD', 'LN%D',
       'SH%D', 'HIPD', 'PWDD', 'PWTD', 'SLAD', 'CHTD', 'CWID', 'RDPD', 'RL1D',
       'RL2D', 'RL3D', 'RL4D', 'RL5D', 'RL6D', 'RL7D', 'RL8D', 'RL9D', 'RL10D',
       'CDAD', 'LDAD', 'SDAD', 'SNW

In [48]:
state = {
        'DAP': current_day,
        'days_since_last_rain': current_soilwat['DaysSinceLastRain'].iloc[0],
        'rainfall_7day': current_soilwat['Rainfall7DaySum'].iloc[0],
        'HarvestIndex' : current_plantgro['HIAD'].iloc[0],
        'phenological_stage': current_plantgro['GSTD'].iloc[0],
        'leaf_area_index': current_plantgro['LAID'].iloc[0],
        'total_biomass': current_plantgro['CWAD'].iloc[0],
        'soil_water_content_0_30cm': get_sw_by_depth(0, 30),
        'soil_water_content_30_60cm': get_sw_by_depth(30, 60),
        'soil_water_content_60_100cm': get_sw_by_depth(60, 100),
        'available_water_fraction': current_soilwat['SWXD'].iloc[0],
        'water_stress_factor': current_plantgro['WSGD'].iloc[0],
        'temperature_avg': current_weather['TAVD'].iloc[0],
        'solar_radiation': current_weather['SRAD'].iloc[0],
        'last_irrigation_amount': irrigation_events[-1].get('irval'),
        'days_since_last_irrigation': days_since_last_irrigation,
        'cumulative_irrigation': current_soilwat['IRRC'].iloc[0],
        'forecast' : dssat_output['SoilWat'][dssat_output['SoilWat']['DAS'] == current_day+1]['DPREC'].iloc[0]
    }
state

{'DAP': 2,
 'days_since_last_rain': np.float64(0.0),
 'rainfall_7day': np.float64(28.0),
 'HarvestIndex': np.float64(0.0),
 'phenological_stage': np.int64(0),
 'leaf_area_index': np.float64(0.0),
 'total_biomass': np.int64(0),
 'soil_water_content_0_30cm': np.float64(0.3426666666666667),
 'soil_water_content_30_60cm': np.float64(0.242),
 'soil_water_content_60_100cm': np.float64(0.242),
 'available_water_fraction': np.int64(30),
 'water_stress_factor': np.float64(0.0),
 'temperature_avg': np.float64(28.7),
 'solar_radiation': np.float64(20.9),
 'last_irrigation_amount': 10.0,
 'days_since_last_irrigation': 2,
 'cumulative_irrigation': np.int64(10),
 'forecast': np.float64(0.0)}

In [49]:
state['days_since_last_rain']

np.float64(0.0)

Of course! Here are the full forms for the DSSAT output variables you listed.

---
### ## 🌦️ Weather (`current_weather`)
* **@YEAR**: Year
* **DOY**: Day of Year
* **DAS**: Days After Sowing
* **PRED**: Photoperiod (hours)
* **DAYLD**: Day Length (hours)
* **TWLD**: Twilight Day Length (for winter wheat) (hours)
* **SRAD**: Solar Radiation (MJ/m²/day)
* **PARD**: Photosynthetically Active Radiation (MJ/m²/day)
* **CLDD**: Cloudiness (%)
* **TMXD**: Maximum Air Temperature (°C)
* **TMND**: Minimum Air Temperature (°C)
* **TAVD**: Average Daily Air Temperature (°C)
* **TDYD**: Average Daytime Temperature (°C)
* **TDWD**: Wet Bulb Temperature (°C)
* **TGAD**: Average Canopy Temperature (°C)
* **TGRD**: Average Root Zone Temperature (°C)
* **WDSD**: Wind Speed (km/day)
* **CO2D**: CO2 Concentration (ppm)
* **VPDF**: Vapor Pressure Deficit (kPa)
* **VPD**: Vapor Pressure (kPa)
* **OZON7**: 7-hour Average Ozone Concentration (ppb)
* **WDATE**: Weather Date (in YYYYDOY format)

---
### ##💧 Soil Water (`current_soilwat`)
* **@YEAR**: Year
* **DOY**: Day of Year
* **DAS**: Days After Sowing
* **SWTD**: Total Soil Water in Profile (mm)
* **SWXD**: Extractable Soil Water in Profile for the plant (mm)
* **ROFC**: Cumulative Runoff (mm)
* **DRNC**: Cumulative Drainage (mm)
* **PREC**: Cumulative Precipitation (mm)
* **IR#C**: Cumulative Number of Irrigations
* **IRRC**: Cumulative Irrigation Amount (mm)
* **DTWT**: Depth to Water Table (cm)
* **MWTD**: Water Stress Factor for Photosynthesis
* **TDFD**: Daily Transpiration (mm)
* **TDFC**: Cumulative Transpiration (mm)
* **ROFD**: Daily Runoff (mm)
* **SW1D** to **SW10**: Soil Water in Layer 1 to 10 (cm³/cm³)
* **DAP**: Days After Planting

---
### ## 🌱 Plant Growth (`current_plantgro`)
* **@YEAR**: Year
* **DOY**: Day of Year
* **DAS**: Days After Sowing
* **DAP**: Days After Planting
* **L#SD**: Number of Live Leaves
* **GSTD**: Growth Stage (code for the plant's development stage)
* **LAID**: Leaf Area Index (m²/m²)
* **LWAD**: Leaf Weight (kg/ha)
* **SWAD**: Stem Weight (kg/ha)
* **GWAD**: Grain Weight (kg/ha)
* **RWAD**: Root Weight (kg/ha)
* **VWAD**: Vegetative Weight (Leaf + Stem) (kg/ha)
* **CWAD**: Canopy Weight (Total above-ground biomass) (kg/ha)
* **G#AD**: Number of Grains (per m² or per plant)
* **GWGD**: Individual Grain Weight (g)
* **HIAD**: Harvest Index
* **PWAD**: Panicle Weight (kg/ha)
* **P#AD**: Panicle Number (per m²)
* **WSPD**: Water Stress Factor for Photosynthesis (0-1 scale)
* **WSGD**: Water Stress Factor for Growth (0-1 scale)
* **NSTD**: Nitrogen Stress Factor for Photosynthesis (0-1 scale)
* **EWSD**: Evaporation Water Stress Factor
* **PST1A**, **PST2A**: Phosphorus Stress Factors
* **KSTD**: Potassium Stress Factor
* **LN%D**: Leaf Nitrogen Concentration (%)
* **SH%D**: Shell Nitrogen Concentration (%)
* **HIPD**: Harvest Index at Physiological Maturity
* **PWDD**: Pod Weight (kg/ha)
* **PWTD**: Tuber Weight (kg/ha)
* **SLAD**: Specific Leaf Area (cm²/g)
* **CHTD**: Canopy Height (cm)
* **CWID**: Canopy Width (cm)
* **RDPD**: Root Depth (cm)
* **RL1D** to **RL10D**: Root Length Density in Layer 1 to 10 (cm/cm³)
* **CDAD**: Daily Change in Canopy Weight (kg/ha/day)
* **LDAD**: Daily Change in Leaf Weight (kg/ha/day)
* **SDAD**: Daily Change in Stem Weight (kg/ha/day)
* **SNW0C**, **SNW1C**: Snow Water Content variables
* **DTTD**: Daily Thermal Time (same as daily GDD) (°C-day)

---
### ## 🍂 Soil Organics (`current_soilorg`)
* **@YEAR**: Year
* **DOY**: Day of Year
* **DAS**: Days After Sowing
* **OMAC**: Cumulative Organic Matter Mineralization (kg/ha)
* **SCDD**: Daily Soil Carbon Change (kg/ha)
* **SOCD**: Soil Organic Carbon in profile (kg/ha)
* **SC0D**, **SCTD**: Soil Carbon in different pools
* **SOMCT**: Total Soil Organic Matter Carbon (t/ha)
* **LCTD**: Litter Carbon (kg C/ha)
* **ONAC**: Cumulative Organic Nitrogen Mineralization (kg N/ha)
* **SNDD**: Daily Soil Nitrogen Change (kg N/ha)
* **SOND**: Soil Organic Nitrogen in profile (kg N/ha)
* **SN0D**, **SNTD**: Soil Nitrogen in different pools
* **SOMNT**: Total Soil Organic Matter Nitrogen (t/ha)
* **LNTD**: Litter Nitrogen (kg N/ha)